In [ ]:
import pyodbc
import pandas as pd

# Conectar ao banco de dados MySQL via ODBC usando DSN
dsn_name = 'vojoana'
conn = pyodbc.connect(f'DSN={dsn_name};')

# Query para buscar os dados
query = """
SELECT ped_spvcodigo, ped_numero, ped_dtemissao, ped_pescodigo,  
       pes_razao, pes_fantasia, ped_pesobruto, ped_stpcodigo
FROM pedidos, pessoa 
WHERE ped_natcodigo IN ('VEN', 'VES', 'VIN', 'VIS')
  AND ped_pescodigo = pes_codigo 
  AND ped_pescodigo <> 1 
  AND ped_stpcodigo = 6 
ORDER BY ped_dtemissao;
"""

# Executar a query e armazenar os resultados em um DataFrame
df = pd.read_sql(query, conn)

# Fechar a conexão
conn.close()

# Converter a coluna de data de emissão para o tipo datetime
df['ped_dtemissao'] = pd.to_datetime(df['ped_dtemissao'])

# Criar uma nova coluna 'mes_ano' para agrupar os pedidos por mês e ano
df['mes_ano'] = df['ped_dtemissao'].dt.to_period('M')

# Calcular o peso médio mensal por cliente
df_grouped = df.groupby(['ped_pescodigo', 'pes_razao', 'pes_fantasia']).agg(
    media_peso_mensal=('ped_pesobruto', 'mean')
).reset_index()

# Função para segmentar os clientes
def segmentar_cliente(media_peso):
    if media_peso >= 400:
        return 'CLIENTE OURO'
    elif 200 <= media_peso < 400:
        return 'CLIENTE PRATA'
    elif 80 <= media_peso < 200:
        return 'CLIENTE BRONZE'
    else:
        return None

# Aplicar a segmentação
df_grouped['segmento'] = df_grouped['media_peso_mensal'].apply(segmentar_cliente)

# Filtrar clientes com peso médio mensal >= 80 kg
df_filtered = df_grouped[df_grouped['segmento'].notnull()]

# Exportar o DataFrame filtrado para um arquivo Excel
df_filtered.to_excel('clientes_segmentados.xlsx', index=False)

print("Arquivo Excel 'clientes_segmentados.xlsx' gerado com sucesso.")


In [ ]:
import pyodbc
import pandas as pd

# Conectar ao banco de dados MySQL via ODBC usando DSN
dsn_name = 'vojoana'
conn = pyodbc.connect(f'DSN={dsn_name};')

# Executar consulta SQL
query = """
SELECT ped_pescodigo, ped_dtemissao, pes_razao, pes_fantasia, ped_pesobruto
FROM pedidos
JOIN pessoa ON ped_pescodigo = pes_codigo
WHERE ped_natcodigo IN ('VEN', 'VES', 'VIN', 'VIS') AND ped_pescodigo <> 1 AND ped_stpcodigo = 6;
"""
df = pd.read_sql(query, conn)
conn.close()

# Preparar dados
df['ped_dtemissao'] = pd.to_datetime(df['ped_dtemissao'])
df['mes_ano'] = df['ped_dtemissao'].dt.to_period('M')

# Agrupar e calcular média mensal de peso por cliente
df_grouped = df.groupby(['ped_pescodigo', 'pes_razao', 'pes_fantasia', 'mes_ano']).agg(total_peso=('ped_pesobruto', 'sum')).reset_index()
df_final = df_grouped.groupby(['ped_pescodigo', 'pes_razao', 'pes_fantasia']).agg(media_peso_mensal=('total_peso', 'mean')).reset_index()

# Segmentar clientes
def segmentar(peso):
    if peso >= 400:
        return 'CLIENTE OURO'
    elif peso >= 200:
        return 'CLIENTE PRATA'
    elif peso >= 80:
        return 'CLIENTE BRONZE'

df_final['segmento'] = df_final['media_peso_mensal'].apply(segmentar)
df_final = df_final[df_final['segmento'].notnull()]

# Exportar para Excel
df_final.to_excel('Clientes_Segmentadosnov.xlsx', index=False)

print("Arquivo Excel 'Clientes_Segmentadosnov.xlsx' gerado com sucesso.")


In [ ]:
import pyodbc
import pandas as pd

# Configurar a conexão ODBC usando o DSN
dsn_name = 'vojoana'
connection = pyodbc.connect(f'DSN={dsn_name}')

# Definir a query
query = """
SELECT
    fng_pescodigo,
    DATEDIFF(fng_dtpagto, fng_dtvencto) AS atraso
FROM
    finang 
WHERE
    fng_tiporecpag = 'R'
    AND fng_situacao = 'L'
"""

# Ler os dados da query para um DataFrame
df = pd.read_sql(query, connection)

# Fechar a conexão
connection.close()

# Agrupar os dados por cliente e calcular o atraso médio
df_grouped = df.groupby('fng_pescodigo', as_index=False)['atraso'].mean()

# Renomear a coluna do atraso médio
df_grouped.rename(columns={'atraso': 'atraso_medio'}, inplace=True)

# Exportar para um arquivo Excel
output_filename = 'atraso_medio_vojoana2.xlsx'
df_grouped.to_excel(output_filename, index=False)

print(f'Arquivo Excel gerado: {output_filename}')


In [ ]:
import pyodbc
import pandas as pd

# Conectar ao banco de dados MySQL via ODBC usando DSN
dsn_name = 'vojoana'
conn = pyodbc.connect(f'DSN={dsn_name};')

# Executar consulta SQL
query = """
SELECT ped_pescodigo, ped_dtemissao, pes_razao, pes_fantasia, ped_pesobruto
FROM pedidos
JOIN pessoa ON ped_pescodigo = pes_codigo
WHERE ped_natcodigo IN ('VEN', 'VES', 'VIN', 'VIS') AND ped_pescodigo <> 1 AND ped_stpcodigo = 6;
"""
df = pd.read_sql(query, conn)
conn.close()

# Preparar dados
df['ped_dtemissao'] = pd.to_datetime(df['ped_dtemissao'])
df['mes_ano'] = df['ped_dtemissao'].dt.to_period('M')

# Agrupar e calcular média mensal de peso por cliente
df_grouped = df.groupby(['ped_pescodigo', 'pes_razao', 'pes_fantasia', 'mes_ano']).agg(total_peso=('ped_pesobruto', 'sum')).reset_index()
df_final = df_grouped.groupby(['ped_pescodigo', 'pes_razao', 'pes_fantasia']).agg(media_peso_mensal=('total_peso', 'mean')).reset_index()

# Segmentar clientes
def segmentar(peso):
    if peso > 599:
        return 'CLIENTE DIAMANTE'
    elif peso >= 400:
        return 'CLIENTE OURO'
    elif peso >= 200:
        return 'CLIENTE PRATA'
    elif peso >= 80:
        return 'CLIENTE BRONZE'
    elif peso > 0:
        return 'CLIENTE PROSPECT'

df_final['segmento'] = df_final['media_peso_mensal'].apply(segmentar)
df_final = df_final[df_final['segmento'].notnull()]

# Exportar para Excel
df_final.to_excel('Clientes_Segmentadosdez.xlsx', index=False)

print("Arquivo Excel 'Clientes_Segmentadosdez.xlsx' gerado com sucesso.")


In [ ]:
import pyodbc
import pandas as pd

# Conectar ao banco de dados MySQL via ODBC usando DSN
dsn_name = 'segvojoana'
conn = pyodbc.connect(f'DSN={dsn_name};')

# Etapa 1: Consulta para obter os dados necessários
query = """
SELECT ped_pescodigo, ped_dtemissao, pes_razao, pes_fantasia, ped_pesobruto
FROM pedidos
JOIN pessoa ON ped_pescodigo = pes_codigo
WHERE ped_natcodigo IN ('VEN', 'VES', 'VIN', 'VIS') AND ped_pescodigo <> 1 AND ped_stpcodigo = 6;
"""
df = pd.read_sql(query, conn)
conn.close()

# Etapa 2: Preparar os dados
df['ped_dtemissao'] = pd.to_datetime(df['ped_dtemissao'])
df['mes_ano'] = df['ped_dtemissao'].dt.to_period('M')

# Agrupar e calcular média mensal de peso por cliente
df_grouped = df.groupby(['ped_pescodigo', 'pes_razao', 'pes_fantasia', 'mes_ano']).agg(total_peso=('ped_pesobruto', 'sum')).reset_index()
df_final = df_grouped.groupby(['ped_pescodigo', 'pes_razao', 'pes_fantasia']).agg(media_peso_mensal=('total_peso', 'mean')).reset_index()

# Segmentar clientes
def segmentar(peso):
    if peso > 599:
        return 4  # CLIENTE DIAMANTE
    elif peso >= 400:
        return 1  # CLIENTE OURO
    elif peso >= 200:
        return 2  # CLIENTE PRATA
    elif peso >= 80:
        return 3  # CLIENTE BRONZE
    elif peso > 0:
        return 5  # CLIENTE PROSPECT

# Adicionar segmento baseado na média de peso mensal
df_final['clc_codigo'] = df_final['media_peso_mensal'].apply(segmentar)

# Filtrar clientes com segmento definido
df_final = df_final[df_final['clc_codigo'].notnull()]

# Etapa 3: Gerar o script SQL
sql_commands = []
for _, row in df_final.iterrows():
    # Gerar comando SQL para atualizar a classificação
    sql = f"UPDATE pessoa SET pes_clccodigo = {row['clc_codigo']} WHERE pes_codigo = {row['ped_pescodigo']};"
    sql_commands.append(sql)

# Salvar comandos SQL em um arquivo
with open('atualizar_classificacao.sql', 'w', encoding='utf-8') as file:
    file.write('\n'.join(sql_commands))

print("Arquivo SQL 'atualizar_classificacao.sql' gerado com sucesso!")


In [ ]:
import pyodbc
import pandas as pd

# Conectar ao banco de dados MySQL via ODBC usando DSN
dsn_name = 'segvojoana'
conn = pyodbc.connect(f'DSN={dsn_name};')

# Etapa 1: Consulta para obter os dados necessários
query = """
SELECT ped_pescodigo, ped_dtemissao, pes_razao, pes_fantasia, ped_pesobruto, pes_clccodigo
FROM pedidos
JOIN pessoa ON ped_pescodigo = pes_codigo
WHERE ped_natcodigo IN ('VEN', 'VES', 'VIN', 'VIS') AND ped_pescodigo <> 1 AND ped_stpcodigo = 6;
"""
df = pd.read_sql(query, conn)

# Consultar classificações
query_classificacoes = """
SELECT clc_codigo, clc_desc
FROM classificapes;
"""
df_classificacoes = pd.read_sql(query_classificacoes, conn)

# Fechar conexão após carregar os dados
conn.close()

# Criar um dicionário de mapeamento de código para descrição
codigo_to_desc = df_classificacoes.set_index('clc_codigo')['clc_desc'].to_dict()

# Etapa 2: Preparar os dados
df['ped_dtemissao'] = pd.to_datetime(df['ped_dtemissao'])
df['mes_ano'] = df['ped_dtemissao'].dt.to_period('M')

# Agrupar e calcular média mensal de peso por cliente
df_grouped = df.groupby(['ped_pescodigo', 'pes_razao', 'pes_fantasia', 'mes_ano']).agg(total_peso=('ped_pesobruto', 'sum')).reset_index()
df_final = df_grouped.groupby(['ped_pescodigo', 'pes_razao', 'pes_fantasia']).agg(media_peso_mensal=('total_peso', 'mean')).reset_index()

# Segmentar clientes
def segmentar(peso):
    if peso > 599:
        return 4  # CLIENTE DIAMANTE
    elif peso >= 400:
        return 1  # CLIENTE OURO
    elif peso >= 200:
        return 2  # CLIENTE PRATA
    elif peso >= 80:
        return 3  # CLIENTE BRONZE
    elif peso > 0:
        return 5  # CLIENTE PROSPECT

# Adicionar segmento baseado na média de peso mensal
df_final['nova_clc_codigo'] = df_final['media_peso_mensal'].apply(segmentar)

# Garantir que `df` tenha valores únicos para `ped_pescodigo`
df_unique = df.drop_duplicates(subset='ped_pescodigo')

# Adicionar descrição da classificação atual e nova
df_final['desc_clc_atual'] = df_final['ped_pescodigo'].map(
    df_unique.set_index('ped_pescodigo')['pes_clccodigo']
).map(codigo_to_desc)
df_final['pes_clccodigo'] = df_final['ped_pescodigo'].map(
    df_unique.set_index('ped_pescodigo')['pes_clccodigo']
)
df_final['desc_clc_nova'] = df_final['nova_clc_codigo'].map(codigo_to_desc)

# Identificar mudanças de classificação
df_final['mudanca'] = df_final.apply(
    lambda row: 'Upgrade' if row['nova_clc_codigo'] < row['pes_clccodigo'] else
                'Downgrade' if row['nova_clc_codigo'] > row['pes_clccodigo'] else
                'Sem alteração',
    axis=1
)

# Excluir clientes sem classificação inicial (nulos) que foram classificados como PROSPECT
df_final = df_final[~((df_final['pes_clccodigo'].isna()) & (df_final['nova_clc_codigo'] == 5))]

# Filtrar apenas clientes com mudanças
df_mudancas = df_final[df_final['mudanca'] != 'Sem alteração']

# Gerar relatório em Excel
df_mudancas[['ped_pescodigo', 'pes_razao', 'pes_fantasia', 'desc_clc_atual', 'desc_clc_nova', 'mudanca']].to_excel(
    'Relatorio_Alteracoes_Classificacao.xlsx', index=False
)

# Gerar script SQL para atualizar as classificações
sql_commands = []
for _, row in df_final.iterrows():
    sql = f"UPDATE pessoa SET pes_clccodigo = {row['nova_clc_codigo']} WHERE pes_codigo = {row['ped_pescodigo']};"
    sql_commands.append(sql)

with open('atualizar_classificacao.sql', 'w', encoding='utf-8') as file:
    file.write('\n'.join(sql_commands))

print("Relatório e script SQL gerados com sucesso!")


In [ ]:
import pyodbc
import pandas as pd

# Conectar ao banco de dados MySQL via ODBC usando DSN
dsn_name = 'segvojoana'
conn = pyodbc.connect(f'DSN={dsn_name};')

# Etapa 1: Consulta para obter os dados necessários
query = """
SELECT ped_pescodigo, ped_dtemissao, pes_razao, pes_fantasia, ped_pesobruto, pes_clccodigo
FROM pedidos
JOIN pessoa ON ped_pescodigo = pes_codigo
WHERE ped_natcodigo IN ('VEN', 'VES', 'VIN', 'VIS') AND ped_pescodigo <> 1 AND ped_stpcodigo = 6;
"""
df = pd.read_sql(query, conn)

# Consultar classificações
query_classificacoes = """
SELECT clc_codigo, clc_desc
FROM classificapes;
"""
df_classificacoes = pd.read_sql(query_classificacoes, conn)

# Fechar conexão após carregar os dados
conn.close()

# Criar um dicionário de mapeamento de código para descrição
codigo_to_desc = df_classificacoes.set_index('clc_codigo')['clc_desc'].to_dict()

# Etapa 2: Preparar os dados
df['ped_dtemissao'] = pd.to_datetime(df['ped_dtemissao'])
df['mes_ano'] = df['ped_dtemissao'].dt.to_period('M')

# Agrupar e calcular média mensal de peso por cliente
df_grouped = df.groupby(['ped_pescodigo', 'pes_razao', 'pes_fantasia', 'mes_ano']).agg(total_peso=('ped_pesobruto', 'sum')).reset_index()
df_final = df_grouped.groupby(['ped_pescodigo', 'pes_razao', 'pes_fantasia']).agg(media_peso_mensal=('total_peso', 'mean')).reset_index()

# Segmentar clientes
def segmentar(peso):
    if peso > 599:
        return 4  # CLIENTE DIAMANTE
    elif peso >= 400:
        return 1  # CLIENTE OURO
    elif peso >= 200:
        return 2  # CLIENTE PRATA
    elif peso >= 80:
        return 3  # CLIENTE BRONZE
    elif peso > 0:
        return 5  # CLIENTE PROSPECT

# Adicionar segmento baseado na média de peso mensal
df_final['nova_clc_codigo'] = df_final['media_peso_mensal'].apply(segmentar)

# Garantir que `df` tenha valores únicos para `ped_pescodigo`
df_unique = df.drop_duplicates(subset='ped_pescodigo')

# Adicionar descrição da classificação atual e nova
df_final['pes_clccodigo'] = df_final['ped_pescodigo'].map(
    df_unique.set_index('ped_pescodigo')['pes_clccodigo']
)
df_final['desc_clc_atual'] = df_final['pes_clccodigo'].map(codigo_to_desc)
df_final['desc_clc_nova'] = df_final['nova_clc_codigo'].map(codigo_to_desc)

# Identificar mudanças de classificação
def identificar_mudanca(row):
    if pd.isna(row['desc_clc_atual']) and row['desc_clc_nova'] == 'CLIENTE PROSPECT':
        return 'Upgrade'  # Novo cliente segmentado como PROSPECT
    elif row['nova_clc_codigo'] < row['pes_clccodigo']:
        return 'Upgrade'
    elif row['nova_clc_codigo'] > row['pes_clccodigo']:
        return 'Downgrade'
    else:
        return 'Sem alteração'

df_final['mudanca'] = df_final.apply(identificar_mudanca, axis=1)

# Filtrar apenas clientes com mudanças relevantes
df_mudancas = df_final[df_final['mudanca'] != 'Sem alteração']

# Gerar relatório em Excel com a média em KG atual
df_mudancas[['ped_pescodigo', 'pes_razao', 'pes_fantasia', 'desc_clc_atual', 'desc_clc_nova', 'media_peso_mensal', 'mudanca']].to_excel(
    'Relatorio_Alteracoes_Classificacao.xlsx', index=False
)

# Gerar script SQL para atualizar as classificações
sql_commands = []
for _, row in df_final.iterrows():
    sql = f"UPDATE pessoa SET pes_clccodigo = {row['nova_clc_codigo']} WHERE pes_codigo = {row['ped_pescodigo']};"
    sql_commands.append(sql)

with open('atualizar_classificacao.sql', 'w', encoding='utf-8') as file:
    file.write('\n'.join(sql_commands))

print("Relatório e script SQL gerados com sucesso!")


In [ ]:
import pyodbc
import pandas as pd

# Conectar ao banco de dados MySQL via ODBC usando DSN
dsn_name = 'segvojoana'
conn = pyodbc.connect(f'DSN={dsn_name};')

# Etapa 1: Consulta para obter os dados necessários
query = """
SELECT ped_pescodigo, ped_dtemissao, pes_razao, pes_fantasia, ped_pesobruto, pes_clccodigo
FROM pedidos
JOIN pessoa ON ped_pescodigo = pes_codigo
WHERE ped_natcodigo IN ('VEN', 'VES', 'VIN', 'VIS') AND ped_pescodigo <> 1 AND ped_stpcodigo = 6;
"""
df = pd.read_sql(query, conn)

# Consultar classificações
query_classificacoes = """
SELECT clc_codigo, clc_desc
FROM classificapes;
"""
df_classificacoes = pd.read_sql(query_classificacoes, conn)

# Fechar conexão após carregar os dados
conn.close()

# Criar um dicionário de mapeamento de código para descrição
codigo_to_desc = df_classificacoes.set_index('clc_codigo')['clc_desc'].to_dict()

# Etapa 2: Preparar os dados
df['ped_dtemissao'] = pd.to_datetime(df['ped_dtemissao'])
df['mes_ano'] = df['ped_dtemissao'].dt.to_period('M')

# Agrupar e calcular média mensal de peso por cliente
df_grouped = df.groupby(['ped_pescodigo', 'pes_razao', 'pes_fantasia', 'mes_ano']).agg(total_peso=('ped_pesobruto', 'sum')).reset_index()
df_final = df_grouped.groupby(['ped_pescodigo', 'pes_razao', 'pes_fantasia']).agg(media_peso_mensal=('total_peso', 'mean')).reset_index()

# Segmentar clientes
def segmentar(peso):
    if peso > 599:
        return 4  # CLIENTE DIAMANTE
    elif peso >= 400:
        return 1  # CLIENTE OURO
    elif peso >= 200:
        return 2  # CLIENTE PRATA
    elif peso >= 80:
        return 3  # CLIENTE BRONZE
    elif peso > 0:
        return 5  # CLIENTE PROSPECT

# Adicionar segmento baseado na média de peso mensal
df_final['nova_clc_codigo'] = df_final['media_peso_mensal'].apply(segmentar)

# Garantir que `df` tenha valores únicos para `ped_pescodigo`
df_unique = df.drop_duplicates(subset='ped_pescodigo')

# Adicionar descrição da classificação atual e nova
df_final['pes_clccodigo'] = df_final['ped_pescodigo'].map(
    df_unique.set_index('ped_pescodigo')['pes_clccodigo']
)
df_final['desc_clc_atual'] = df_final['pes_clccodigo'].map(codigo_to_desc)
df_final['desc_clc_nova'] = df_final['nova_clc_codigo'].map(codigo_to_desc)

# Identificar mudanças de classificação
def identificar_mudanca(row):
    # Caso o cliente passe de sem classificação para qualquer classificação
    if pd.isna(row['desc_clc_atual']) and pd.notna(row['desc_clc_nova']):
        return 'Upgrade'  # Novo cliente segmentado
    elif row['nova_clc_codigo'] < row['pes_clccodigo']:
        return 'Upgrade'
    elif row['nova_clc_codigo'] > row['pes_clccodigo']:
        return 'Downgrade'
    else:
        return 'Sem alteração'

df_final['mudanca'] = df_final.apply(identificar_mudanca, axis=1)

# Filtrar apenas clientes com mudanças relevantes
df_mudancas = df_final[df_final['mudanca'] != 'Sem alteração']

# Gerar relatório em CSV com a vírgula como separador decimal
df_mudancas[['ped_pescodigo', 'pes_razao', 'pes_fantasia', 'desc_clc_atual', 'desc_clc_nova', 'media_peso_mensal', 'mudanca']].to_csv(
    'Relatorio_Alteracoes_Classificacao.csv', index=False, sep=';', decimal=',', encoding='utf-8'
)

# Gerar script SQL para atualizar as classificações
sql_commands = []
for _, row in df_final.iterrows():
    sql = f"UPDATE pessoa SET pes_clccodigo = {row['nova_clc_codigo']} WHERE pes_codigo = {row['ped_pescodigo']};"
    sql_commands.append(sql)

with open('atualizar_classificacao.sql', 'w', encoding='utf-8') as file:
    file.write('\n'.join(sql_commands))

print("Relatório em CSV e script SQL gerados com sucesso!")


In [ ]:
import pyodbc
import pandas as pd

# Conectar ao banco de dados MySQL via ODBC usando DSN
dsn_name = 'segvojoana'
conn = pyodbc.connect(f'DSN={dsn_name};')

# Etapa 1: Consulta para obter os dados necessários
query = """
SELECT 
    ped.ped_pescodigo, 
    ped.ped_dtemissao, 
    pes.pes_razao, 
    pes.pes_fantasia, 
    ped.ped_pesobruto, 
    pes.pes_clccodigo, 
    cli.cli_cclcodigoe, 
    ccl.ccl_desc AS carteira_cliente
FROM 
    pedidos ped
JOIN 
    pessoa pes ON ped.ped_pescodigo = pes.pes_codigo
JOIN 
    cliente cli ON pes.pes_codigo = cli.cli_pescodigo
LEFT JOIN 
    carteiracli ccl ON cli.cli_cclcodigoe = ccl.ccl_codigo
WHERE 
    ped.ped_natcodigo IN ('VEN', 'VES', 'VIN', 'VIS') 
    AND ped.ped_pescodigo <> 1 
    AND ped.ped_stpcodigo = 6;
"""
df = pd.read_sql(query, conn)

# Consultar classificações
query_classificacoes = """
SELECT clc_codigo, clc_desc
FROM classificapes;
"""
df_classificacoes = pd.read_sql(query_classificacoes, conn)

# Fechar conexão após carregar os dados
conn.close()

# Criar um dicionário de mapeamento de código para descrição
codigo_to_desc = df_classificacoes.set_index('clc_codigo')['clc_desc'].to_dict()

# Etapa 2: Preparar os dados
df['ped_dtemissao'] = pd.to_datetime(df['ped_dtemissao'])
df['mes_ano'] = df['ped_dtemissao'].dt.to_period('M')

# Agrupar e calcular média mensal de peso por cliente
df_grouped = df.groupby(['ped_pescodigo', 'pes_razao', 'pes_fantasia', 'mes_ano']).agg(total_peso=('ped_pesobruto', 'sum')).reset_index()
df_final = df_grouped.groupby(['ped_pescodigo', 'pes_razao', 'pes_fantasia']).agg(media_peso_mensal=('total_peso', 'mean')).reset_index()

# Segmentar clientes
def segmentar(peso):
    if peso > 599:
        return 4  # CLIENTE DIAMANTE
    elif peso >= 400:
        return 1  # CLIENTE OURO
    elif peso >= 200:
        return 2  # CLIENTE PRATA
    elif peso >= 80:
        return 3  # CLIENTE BRONZE
    elif peso > 0:
        return 5  # CLIENTE PROSPECT

# Adicionar segmento baseado na média de peso mensal
df_final['nova_clc_codigo'] = df_final['media_peso_mensal'].apply(segmentar)

# Garantir que `df` tenha valores únicos para `ped_pescodigo`
df_unique = df.drop_duplicates(subset='ped_pescodigo')

# Adicionar descrição da classificação atual e nova
df_final['pes_clccodigo'] = df_final['ped_pescodigo'].map(
    df_unique.set_index('ped_pescodigo')['pes_clccodigo']
)
df_final['desc_clc_atual'] = df_final['pes_clccodigo'].map(codigo_to_desc)
df_final['desc_clc_nova'] = df_final['nova_clc_codigo'].map(codigo_to_desc)

# Identificar mudanças de classificação
def identificar_mudanca(row):
    # Caso o cliente passe de sem classificação para qualquer classificação
    if pd.isna(row['desc_clc_atual']) and pd.notna(row['desc_clc_nova']):
        return 'Upgrade'  # Novo cliente segmentado
    elif row['nova_clc_codigo'] < row['pes_clccodigo']:
        return 'Upgrade'
    elif row['nova_clc_codigo'] > row['pes_clccodigo']:
        return 'Downgrade'
    else:
        return 'Sem alteração'

df_final['mudanca'] = df_final.apply(identificar_mudanca, axis=1)

# Filtrar apenas clientes com mudanças relevantes
df_mudancas = df_final[df_final['mudanca'] != 'Sem alteração']

# Adicionar informações da carteira ao relatório
df_mudancas = df_mudancas.merge(
    df[['ped_pescodigo', 'carteira_cliente']].drop_duplicates(),
    on='ped_pescodigo',
    how='left'
)

# Gerar relatório em CSV com a vírgula como separador decimal
df_mudancas[['ped_pescodigo', 'pes_razao', 'pes_fantasia', 'carteira_cliente', 'desc_clc_atual', 'desc_clc_nova', 'media_peso_mensal', 'mudanca']].to_csv(
    'Relatorio_Alteracoes_Classificacao.csv', index=False, sep=';', decimal=',', encoding='utf-8'
)

# Gerar script SQL para atualizar as classificações
sql_commands = []
for _, row in df_final.iterrows():
    sql = f"UPDATE pessoa SET pes_clccodigo = {row['nova_clc_codigo']} WHERE pes_codigo = {row['ped_pescodigo']};"
    sql_commands.append(sql)

with open('atualizar_classificacao.sql', 'w', encoding='utf-8') as file:
    file.write('\n'.join(sql_commands))

print("Relatório em CSV e script SQL gerados com sucesso!")


In [ ]:
from datetime import datetime

# Obter a data atual
data_atualizacao = datetime.now().strftime('%Y-%m-%d')

# Adicionar a coluna de data de atualização
df_final['data_atualizacao'] = data_atualizacao

# Filtrar apenas clientes com mudanças relevantes
df_mudancas = df_final[df_final['mudanca'] != 'Sem alteração']

# Adicionar informações da carteira ao relatório
df_mudancas = df_mudancas.merge(
    df[['ped_pescodigo', 'carteira_cliente']].drop_duplicates(),
    on='ped_pescodigo',
    how='left'
)

# Gerar relatório em CSV com a vírgula como separador decimal
df_mudancas[['ped_pescodigo', 'pes_razao', 'pes_fantasia', 'carteira_cliente', 'desc_clc_atual', 'desc_clc_nova', 'media_peso_mensal', 'mudanca', 'data_atualizacao']].to_csv(
    'Relatorio_Alteracoes_Classificacao.csv', index=False, sep=';', decimal=',', encoding='utf-8'
)

# Gerar script SQL para atualizar as classificações
sql_commands = []
for _, row in df_final.iterrows():
    sql = f"UPDATE pessoa SET pes_clccodigo = {row['nova_clc_codigo']}, data_atualizacao = '{data_atualizacao}' WHERE pes_codigo = {row['ped_pescodigo']};"
    sql_commands.append(sql)

with open('atualizar_classificacao.sql', 'w', encoding='utf-8') as file:
    file.write('\n'.join(sql_commands))

print("Relatório em CSV e script SQL gerados com sucesso!")


In [1]:
import pyodbc
import pandas as pd
from datetime import datetime

# Conectar ao banco de dados MySQL via ODBC usando DSN
dsn_name = 'segvojoana'
conn = pyodbc.connect(f'DSN={dsn_name};')

# Etapa 1: Consulta para obter os dados necessários
query = """
SELECT 
    ped.ped_pescodigo, 
    ped.ped_dtemissao, 
    pes.pes_razao, 
    pes.pes_fantasia, 
    ped.ped_pesobruto, 
    pes.pes_clccodigo, 
    cli.cli_cclcodigoe, 
    ccl.ccl_desc AS carteira_cliente
FROM 
    pedidos ped
JOIN 
    pessoa pes ON ped.ped_pescodigo = pes.pes_codigo
JOIN 
    cliente cli ON pes.pes_codigo = cli.cli_pescodigo
LEFT JOIN 
    carteiracli ccl ON cli.cli_cclcodigoe = ccl.ccl_codigo
WHERE 
    ped.ped_natcodigo IN ('VEN', 'VES', 'VIN', 'VIS') 
    AND ped.ped_pescodigo <> 1 
    AND ped.ped_stpcodigo = 6
    AND ped.ped_dtemissao >= DATE_SUB(CURRENT_DATE, INTERVAL 12 MONTH);
"""
df = pd.read_sql(query, conn)

# Consultar classificações
query_classificacoes = """
SELECT clc_codigo, clc_desc
FROM classificapes;
"""
df_classificacoes = pd.read_sql(query_classificacoes, conn)

# Fechar conexão após carregar os dados
conn.close()

# Criar um dicionário de mapeamento de código para descrição
codigo_to_desc = df_classificacoes.set_index('clc_codigo')['clc_desc'].to_dict()

# Etapa 2: Preparar os dados
df['ped_dtemissao'] = pd.to_datetime(df['ped_dtemissao'])
df['mes_ano'] = df['ped_dtemissao'].dt.to_period('M')

# Agrupar e calcular média mensal de peso por cliente
df_grouped = df.groupby(['ped_pescodigo', 'pes_razao', 'pes_fantasia', 'mes_ano']).agg(total_peso=('ped_pesobruto', 'sum')).reset_index()
df_final = df_grouped.groupby(['ped_pescodigo', 'pes_razao', 'pes_fantasia']).agg(media_peso_mensal=('total_peso', 'mean')).reset_index()

# Segmentar clientes
def segmentar(peso):
    if peso > 599:
        return 'CLIENTE A'
    elif peso >= 400:
        return 'CLIENTE B'
    elif peso >= 200:
        return 'CLIENTE C'
    elif peso >= 80:
        return 'CLIENTE D'
    elif peso > 0:
        return 'CLIENTE E'

# Adicionar segmento baseado na média de peso mensal
df_final['desc_clc_nova'] = df_final['media_peso_mensal'].apply(segmentar)

# Garantir que `df` tenha valores únicos para `ped_pescodigo`
df_unique = df.drop_duplicates(subset='ped_pescodigo')

# Adicionar descrição da classificação atual
df_final['pes_clccodigo'] = df_final['ped_pescodigo'].map(
    df_unique.set_index('ped_pescodigo')['pes_clccodigo']
)
df_final['desc_clc_atual'] = df_final['pes_clccodigo'].map(codigo_to_desc)

# Definir hierarquia explícita
hierarquia = ['CLIENTE E', 'CLIENTE D', 'CLIENTE C', 'CLIENTE B', 'CLIENTE A']

# Identificar mudanças de classificação
def identificar_mudanca(row):
    atual_pos = hierarquia.index(row['desc_clc_atual']) if row['desc_clc_atual'] in hierarquia else -1
    nova_pos = hierarquia.index(row['desc_clc_nova']) if row['desc_clc_nova'] in hierarquia else -1

    if atual_pos == -1 and nova_pos != -1:
        return 'Upgrade'
    elif nova_pos > atual_pos:
        return 'Upgrade'
    elif nova_pos < atual_pos:
        return 'Downgrade'
    else:
        return 'Sem alteração'

df_final['mudanca'] = df_final.apply(identificar_mudanca, axis=1)

# Filtrar apenas clientes com mudanças relevantes
df_mudancas = df_final[df_final['mudanca'] != 'Sem alteração']

# Adicionar a data da atualização
data_atualizacao = datetime.now().strftime('%Y-%m-%d')
df_mudancas['data_atualizacao'] = data_atualizacao

# Adicionar informações da carteira ao relatório
df_mudancas = df_mudancas.merge(
    df[['ped_pescodigo', 'carteira_cliente']].drop_duplicates(),
    on='ped_pescodigo',
    how='left'
)

# Gerar relatório em CSV com a vírgula como separador decimal
df_mudancas[['ped_pescodigo', 'pes_razao', 'pes_fantasia', 'carteira_cliente', 'desc_clc_atual', 'desc_clc_nova', 'media_peso_mensal', 'mudanca', 'data_atualizacao']].to_csv(
    'Relatorio_Alteracoes_Classificacaoxx.csv', index=False, sep=';', decimal=',', encoding='utf-8'
)

# Gerar script SQL para atualizar as classificações
sql_commands = []
for _, row in df_final.iterrows():
    sql = f"UPDATE pessoa SET pes_clccodigo = (SELECT clc_codigo FROM classificapes WHERE clc_desc = '{row['desc_clc_nova']}') WHERE pes_codigo = {row['ped_pescodigo']};"
    sql_commands.append(sql)

with open('atualizar_classificacaoxx.sql', 'w', encoding='utf-8') as file:
    file.write('\n'.join(sql_commands))

print("Relatório em CSV e script SQL gerados com sucesso!")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\Gabriel D. Barros\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Gabriel D. Barros\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Gabriel D. Barros\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\Gabriel D. Barros\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start(

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\Gabriel D. Barros\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Gabriel D. Barros\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Gabriel D. Barros\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\Gabriel D. Barros\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start(

AttributeError: _ARRAY_API not found

C:\Users\Gabriel D. Barros\AppData\Local\Temp\ipykernel_39524\956150992.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Relatório em CSV e script SQL gerados com sucesso!


C:\Users\Gabriel D. Barros\AppData\Local\Temp\ipykernel_39524\956150992.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_classificacoes = pd.read_sql(query_classificacoes, conn)
C:\Users\Gabriel D. Barros\AppData\Local\Temp\ipykernel_39524\956150992.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mudancas['data_atualizacao'] = data_atualizacao
